In [1]:
from cntools.datasets import load_ct_order, prep_crc, prep_crc_ori, Dataset
from cntools.identification import CC, CFIDF, CNE, Spatial_LDA
from cntools.smoothing import NaiveSmooth, HMRF
from cntools.utils import cns_info
import os
import pandas as pd
import pickle

In [16]:
# Download CRC, T2D, and HLT data
# os.system('gdown --folder https://drive.google.com/drive/folders/1eC3ahSIzCZ5vq_bynSl3IX7V9w_bQgg8?usp=share_link')

In [9]:
# Extract and save original CNs
# df, cns = prep_crc_ori(pd.read_csv('data/CRC/CRC_clusters_neighborhoods_markers.csv'))
# ct_order = load_ct_order('data/CRC/ct_order.json') + ['Dirt']
# ds = Dataset(df, ct_order)

# df.to_csv('data/CRC/CRC_df_ori.csv', index=False)
# pickle.dump(ds, open(f'data/CRC/CRC_ds_ori.pkl', 'wb'))
# pickle.dump(cns, open(f'cn/CRC/CC_ori/cns.pkl', 'wb'))

In [11]:
# Extract and save original CNs without "dirt" cell type
# df, cns = prep_crc_ori(pd.read_csv('data/CRC/CRC_clusters_neighborhoods_markers.csv'), no_dirt=True)
# ct_order = load_ct_order('data/CRC/ct_order.json')
# ds = Dataset(df, ct_order)

# df.to_csv('data/CRC/CRC_df_ori_nodirt.csv', index=False)
# pickle.dump(ds, open(f'data/CRC/CRC_ds_ori_nodirt.pkl', 'wb'))
# pickle.dump(cns, open(f'cn/CRC/CC_ori/cns_nodirt.pkl', 'wb'))

In [11]:
# Load dataset
df = prep_crc(pd.read_csv('data/CRC/CRC_clusters_neighborhoods_markers.csv'))
ct_order = load_ct_order('data/CRC/ct_order.json')
ds = Dataset(df, ct_order)

In [ ]:
# Save processed dataframe and dataset object
df.to_csv('data/CRC/CRC_df.csv', index=False)
pickle.dump(ds, open(f'data/CRC/CRC_ds.pkl', 'wb'))

In [ ]:
# identifier = CC(n_cns=9, m=8)
# identifier = CFIDF(n_cns=9, eps=32, r=0.8)
identifier = CNE(n_cns=9, perp=15, lam=0.25)
# identifier = Spatial_LDA(n_cns=9, eps=50, b=0.025)

cns = identifier.fit(ds)

In [ ]:
smoother = NaiveSmooth(ds=ds, n_cns=identifier.n_cns, feats=identifier.feats, s=3)
cns_smoothed_naive = smoother.fit(cns)

In [ ]:
smoother = HMRF(ds=ds, n_cns=identifier.n_cns, eps=40, beta=9)
cns_smoothed_hmrf = smoother.fit(cns)

In [8]:
out_dir = 'cn/CRC/CNE'
os.makedirs(out_dir, exist_ok=True)
pickle.dump(cns, open(f'{out_dir}/cns_n_cns=9_perp=15_lam=0.25.pkl', 'wb'))
pickle.dump(cns_smoothed_naive, open(f'{out_dir}/cns_n_cns=9_perp=15_lam=0.25_naive.pkl', 'wb'))
pickle.dump(cns_smoothed_hmrf, open(f'{out_dir}/cns_n_cns=9_perp=15_lam=0.25_hmrf.pkl', 'wb'))